In [23]:
import pandas as pd
import numpy as np
from math import log,exp
import datetime
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import logging

import warnings
warnings.filterwarnings('ignore')

In [24]:
class dateCalculator(object):    
    def __init__(self, path):
        temp= pd.read_fwf(path,header=None)
        temp.columns=['holidays']
        self.holi = list(map(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'), temp['holidays']))
    
    @property
    def holidays(self):
        return self.holi

    def val_bus_day(self, d):
        temp = d
        while temp in self.holi or temp.weekday() == 5 or temp.weekday() == 6:
            temp += datetime.timedelta(days=1)
        if d.month!=temp.month:
            temp = d
            while temp in self.holi or temp.weekday() == 5 or temp.weekday() == 6:
                temp -= datetime.timedelta(days=1)
        return temp
    
    def next_depo_expiry(self,x,increment):
        if increment[4]=="M":
            year = x.year
            day = x.day
            m = x.month + int(increment[3])   
            if m > 12:
                m %= 12
                year += 1

            if x.day == monthrange(year, x.month)[1]:
                temp = datetime.datetime(year, m, monthrange(year, m)[1])

            else:
                temp = datetime.datetime(year, m, x.day)
    
        elif increment[4]=="W":
            temp = datetime.timedelta(weeks = int(increment[3])) + x 
        else:
            temp = datetime.timedelta(days = int(increment[3])) + x 
        return self.val_bus_day(temp)
    
    def spot_date(self,d):
        d += datetime.timedelta(days=1)
        d = self.val_bus_day(d)
        d += datetime.timedelta(days=1)
        d = self.val_bus_day(d)
        return d
    
    def ticker_month(self, tickers, s): # s control the year
        dates = []
        next_year = 0
        for ticker in tickers:
            y = s.year - (s.year % 10)
            if ticker[2:]=="H0":
                y += 10
            else: y += int(ticker[-1])

            switcher={'H':3,'M':6,'U':9, 'Z':12}
            m = switcher[ticker[2]]
            d = 1
            s = datetime.datetime(y, m, d)

            switcher2={0:16, 1:15, 2:14, 3:20, 4:19, 5:18, 6:17}
            s += datetime.timedelta(days = switcher2[s.weekday()])
            dates.append(self.val_bus_day(s))
        return dates.copy()

    @staticmethod
    def next_third_wed(s):
        s += datetime.timedelta(days = 91)
        if s.weekday() == 2:
            if 15 <= s.day <= 21:
                return s
            elif s.day < 15:
                return s + datetime.timedelta(days = 7)
            else:
                return s - datetime.timedelta(days = 7)
        return False
    
    @classmethod
    def all_futures_date(cls,s):
        d = cls.ticker_month(futures[0][0], s)
        lst_f = [cls.val_bus_day(d)]
        for i in futures.iloc[1:,0]:
            lst_f.append(cls.val_bus_day(cls.next_third_wed(lst_f[-1])))
        return lst_f
    


In [25]:
class USDYieldCurve(object):
    def __init__(self,*args): # 'depoRates.txt', 'futuresPrices.txt', 'tradeDate.txt', 'holidayCalendar.txt'
        # ----------------------------- read files ---------------------------------------------
        if len(args) == 4:
            # -------------------- get depo and futures and rename their columns ---------------
            self.__depos = pd.read_fwf(args[0],header=None)

            self.__futures = pd.read_fwf(args[1],header=None)

            # -------------------- get holiday, traded date, and spot date ---------------------
            # create dateCalculator object and read holidayCalendar 
            self.__holiday = dateCalculator(args[3])
        
            # read trade day from file
            self.__trade_date = datetime.datetime.strptime(pd.read_fwf(args[2],header=None)[0][0],'%Y-%m-%d') 

            # calculate the spot date which is two days later than trade day
            self.__spot_date = self.__holiday.spot_date(self.__trade_date) 
            
        elif len(args) == 2:
            self.__holiday = dateCalculator(args[1])
            
            dc= pd.read_excel(args[0],sheetname='Discount Curve',header=None)
            self.__spot_date = str(dc.iloc[0,1]).split()[0]
            self.__spot_date = datetime.datetime.strptime(self.__spot_date,'%Y-%m-%d')
            
            self.__depos = pd.read_excel(args[0],sheetname='depoRates',header=None)[2:4]
            self.__depos.reset_index(drop=True,inplace=True)
            
            self.__futures = pd.read_excel(args[0],sheetname='Futures').iloc[1:,1:]
            self.__futures.reset_index(drop=True,inplace=True)
                  
        else:
            logging.error('Cannot build curve from given inputs')
        
        # ---------------------------- transformation -----------------------------------------
        
        self.__depos.columns=['code','rate']
        self.__depos['rate'] /=100
        self.__futures.columns = ['code','price']
        
        # convert codes in depos and futures to valide business dates as expiry dates
        # 将USD*M,ED**翻译出来, 作出YCtestcase “Discount Curve” 表
        self.__depos['expiry'] = [self.__holiday.next_depo_expiry(self.__spot_date,i) for i in self.__depos['code']]
        self.__futures['expiry'] = self.__holiday.ticker_month(self.__futures['code'],self.__spot_date)

        # --------------------- 下面三行代码作出C,D两列 ---------------------------------------
        # get futures implied rate
        self.futures_rate()

        # calculating the discount factors of depo and futures
        self.df_depos()
        self.df_futures()

        # get yield curve by combining depo and futures dataframe
        self.__yield_curve = pd.concat([self.__depos, self.__futures]).drop(['price'],axis=1)
        self.__yield_curve = self.__yield_curve.sort_values(by="expiry" , ascending=True) 
        self.__yield_curve = self.__yield_curve.reset_index(drop=True)
        
        
    @property
    def holiday(self):
        return self.__holiday
    @property
    def depos(self):
        return self.__depos
    @property
    def futures(self):
        return self.__futures
    @property
    def trade_date(self):
        return self.__trade_date
    @property
    def spot_date(self):
        return self.__spot_date
    @property
    def yield_curve(self):
        return self.__yield_curve
    
    def futures_rate(self):
        self.__futures['rate']=1-self.__futures['price']/100
    
    def df_depos(self):
        self.__depos['df']=[1/(1+self.__depos['rate'][ind]*\
                               ((self.__depos['expiry'][ind] - self.__spot_date).days)/360) \

                            for ind in self.__depos.index]
    
    def df_futures(self):
        lnA=log(self.__depos['df'][0])
        lnB=log(self.__depos['df'][1])
        self.__futures['df'] = np.nan
        self.__futures['df'][0]=exp(lnA+(lnB-lnA)*(self.__futures['expiry'][0]-self.__depos['expiry'][0]).days/
                                    (self.__depos['expiry'][1]-self.__depos['expiry'][0]).days)
        
        for i in range(1, len(self.__futures)):
            self.__futures['df'][i]=self.__futures['df'][i-1]/(1+self.__futures['rate'][i-1]*
                                     (self.__futures['expiry'][i]-self.__futures['expiry'][i-1]).days/360)
    
    def getDftoDate(self, d1):
        d1 = datetime.datetime.strptime(d1,'%Y-%m-%d')
        for i in range(len(self.__yield_curve)-1):
            lower = self.__yield_curve['expiry'][i]
            upper = self.__yield_curve['expiry'][i+1]
            if lower < d1 < upper:
                return exp(log(self.__yield_curve['df'][i]) + (d1 - lower).days / (upper - lower).days *
                        (log(self.__yield_curve['df'][i+1]) - log(self.__yield_curve['df'][i])))
 
    
    def getFwdRate(self,d1,d2):
        days = (datetime.datetime.strptime(d2,'%Y-%m-%d') - datetime.datetime.strptime(d1,'%Y-%m-%d')).days
        fwd_rate = 360 / days * (self.getDftoDate(d1)/self.getDftoDate(d2)-1)
        
        return fwd_rate

# Test case 1

## get the holiday calendar for 2015/01/01 - 2019/01/01

we need holiday for those years for us to do the further calculations

In [26]:
# pip install holidays

In [27]:
import holidays
holidayCalendar = []

for year in ([2015,2016,2017,2018]):
    for date in holidays.UnitedStates(years=year).items():
        holidayCalendar.append(str(date[0]))
holidayCalendar

with open("holidayCalendar2015to2019.txt",'w') as f:
    for i in holidayCalendar:
        f.write(i)
        f.write('\n')

## Start Calculation

In [28]:
usdCurve2015 = USDYieldCurve('depoRates.txt', 'futuresPrices.txt', 'tradeDate.txt', 'holidayCalendar2015to2019.txt')
print(usdCurve2015.trade_date)
print(usdCurve2015.spot_date)
usdCurve2015.depos

2015-04-22 00:00:00
2015-04-24 00:00:00


,code,rate,expiry,df
0,USD1M,0.001780,2015-05-26,0.999842
1,USD2M,0.002231,2015-06-24,0.999622


In [29]:
usdCurve2015.yield_curve

,code,df,expiry,rate
0,USD1M,0.999842,2015-05-26,0.001780
1,EDM5,0.999675,2015-06-17,0.003350
2,USD2M,0.999622,2015-06-24,0.002231
3,EDU5,0.998829,2015-09-16,0.004900
4,EDZ5,0.997594,2015-12-16,0.006700
5,EDH6,0.995907,2016-03-16,0.008600
6,EDM6,0.993747,2016-06-15,0.010600
7,EDU6,0.990887,2016-09-21,0.012600
8,EDZ6,0.987742,2016-12-21,0.017200
9,EDH7,0.983793,2017-03-15,0.018800


In [30]:
d1 = '2015-12-01'
d2 = '2016-02-01'
print(usdCurve2015.getDftoDate(d1))
print(usdCurve2015.getDftoDate(d2))
print(usdCurve2015.getFwdRate(d1,d2))

0.9977972618842236
0.9967221933967778
0.006262861602025028


# Test case 2

## Extract Information from YCtestcaseWith2019Spot.xlsm and Save as txt files as above

In [35]:
usdCurve2019 = USDYieldCurve("YCtestcaseWith2019Spot.xlsm",'holidayCalendar.txt')
sd = usdCurve2019.spot_date
sd = datetime.datetime.strftime(sd,'%Y-%m-%d')
sd

'2019-04-25'

In [32]:
# in the class, I select USD1M and USD2M from ticker
usdCurve2019.depos 

,code,rate,expiry,df
0,USD1M,0.001780,2019-05-28,0.999837
1,USD2M,0.002231,2019-06-25,0.999622


In [33]:
usdCurve2019.yield_curve

,code,df,expiry,rate
0,USD1M,0.999837,2019-05-28,0.001780
1,EDM9,0.999668,2019-06-19,0.004350
2,USD2M,0.999622,2019-06-25,0.002231
3,EDU9,0.998570,2019-09-18,0.005900
4,EDZ9,0.997083,2019-12-18,0.007700
5,EDH0,0.995146,2020-03-18,0.009600
6,EDM0,0.992737,2020-06-17,0.011600
7,EDU0,0.989835,2020-09-16,0.013600
8,EDZ0,0.986444,2020-12-16,0.018200
9,EDH1,0.981926,2021-03-17,0.019800


In [34]:
d1 = '2019-12-02'
d2 = '2020-02-3'

print(usdCurve2019.getDftoDate(d1))
print(usdCurve2019.getDftoDate(d2))
print(usdCurve2019.getFwdRate(d1,d2))

0.9973443728908207
0.9960822075173731
0.007240741284329896
